Compare speakers

In [ ]:
import math
import pandas as pd
import altair as alt
import numpy as np
from src.spinorama.analysis import sound_power
from src.spinorama.load import parse_all_speakers, parse_graphs_speaker, graph_melt
from src.spinorama.graph import graph_params_default

import datas.metadata as metadata
df = parse_all_speakers(metadata.speakers_info, None)
# print(df)

In [ ]:
df.keys()

In [ ]:
df['Adam S2V']['ASR']['default']['CEA2034'].head()

In [ ]:
def normalize(df):
    on = df[df.Measurements == 'On Axis']
    mean = np.mean(on.loc[(on.Freq>500) & (on.Freq<10000)].dB)
    
    offset = 0
    if 'DI offset' in df.Measurements.unique():
        offset = np.mean(df[df.Measurements == 'DI offset'].dB)
    else:
        offset = mean
        
    # print(mean, offset, df.Measurements.unique())
    # print(df[df.Measurements == 'Sound Power DI'].head())    
    for measurement in ('On Axis', 'Listening Window', 'Sound Power', 'Early Reflections'):
        df.loc[df.Measurements == measurement, 'dB'] -= mean
        
    for measurement in ('Sound Power DI', 'Early Reflections DI', 'DI offset'):
        df.loc[df.Measurements == measurement, 'dB'] -= offset
        
    return df

def pprint(df):
    for m in df.Measurements.unique():
        min = np.min(df[df.Measurements == m].dB)
        max = np.max(df[df.Measurements == m].dB)
        print('{0} {1} {2}'.format(min, max, m))
        
# adam = df['Adam S2V']['ASR']['default']['CEA2034']
# pprint(adam)
# norm_adam = normalize(adam)
# pprint(norm_adam)
# print(t[t.Measurements == 'Sound Power DI'].head()) 
# print(t[t.Measurements == 'On Axis'].head())

In [ ]:
def resample(df, target_size):
    len_freq = df.shape[0]
    if len_freq > 2*target_size:
        roll = int(len_freq/target_size)
        sampled = df.loc[df.Freq.rolling(roll).max()[1::roll].index,:]
        return sampled
    return df

def augment(dfa, name):
    # print(name)
    namearray = [name for i in range(0,len(dfa))]
    dfa['Speaker'] = name
    return dfa

source = pd.concat( 
    [augment(normalize(resample(df[k][o]['default']['CEA2034'], 400)), k+' - '+o)
         for k in sorted(df.keys()) for o in df[k].keys() if 'CEA2034' in df[k][o]['default']]
)

In [ ]:
source.shape
source.head()

In [ ]:
speaker1 = 'KEF LS50 - ASR'
speaker2 = 'KEF LS50 - Princeton'

nearest = alt.selection(
    type='single',
    nearest=True,
    on='mouseover',
    fields=['Freq'],
    empty='none')

def compare(df):
    speakers = df.Speaker.unique()
    input_dropdown1 = alt.binding_select(options=[s for s in speakers])
    selection1 = alt.selection_single(
        fields=['Speaker'],
        bind=input_dropdown1,
        name='Select right ',
        init={'Speaker': speaker1})
    input_dropdown2 = alt.binding_select(options=[s for s in speakers])
    selection2 = alt.selection_single(
        fields=['Speaker'],
        bind=input_dropdown2,
        name='Select left ',
        init={'Speaker': speaker2})
    selectorsMeasurements = alt.selection_multi(
        fields=['Measurements'],
        bind='legend')
    scales = alt.selection_interval(bind='scales')

    x = alt.X('Freq:Q', scale=alt.Scale(type="log", domain=[20,20000], nice=False))
    yl = alt.Y('dB:Q',  scale=alt.Scale(zero=False, domain=[-40,10]))
    yr = alt.Y('dB:Q',  scale=alt.Scale(zero=False, domain=[0,30]))
    col = alt.Color('Measurements', type='nominal', sort=None)
    opa = alt.condition(selectorsMeasurements, alt.value(1), alt.value(0.2))
    
    line = alt.Chart(df).mark_line(clip=True).transform_filter(
        alt.FieldOneOfPredicate(
            field='Measurements', oneOf=['On Axis', 'Listening Window', 'Early Reflections', 'Sound Power'])
    ).encode(x, yl, col, opacity=opa)
    
    di = alt.Chart(df).mark_line(clip=True).transform_filter(
        alt.FieldOneOfPredicate(
            field='Measurements', oneOf=['Early Reflections DI', 'Sound Power DI'])
    ).encode(x, yr, col, opacity=opa)
    
    spin = (line+di).resolve_scale(y='independent')

    g_speaker1 = spin.add_selection(selection1).transform_filter(selection1)
    g_speaker2 = spin.add_selection(selection2).transform_filter(selection2)

    summed = alt.layer(g_speaker2, g_speaker1
    ).add_selection(selectorsMeasurements#).add_selection(scales
    ).properties(width=780, height=300)#.interactive()
             
    sidebyside = alt.hconcat(
        g_speaker1.properties(width=340, height=150), 
        g_speaker2.properties(width=340, height=150)
    ).add_selection(selectorsMeasurements)#.add_selection(scales)
    
    return sidebyside, summed

g1, g2 = compare(source)
g1 & g2